In [ ]:
import weave

In [ ]:
from weave import syndata

In [ ]:
data = weave.save(syndata.random_metrics(100, 100))
#data

In [ ]:
from weave import ops_arrow
from weave.ecosystem import wandb
import datetime
import random
random.seed(1)

min_x = datetime.datetime(2023, 3, 22, 11, 0, 0)
max_x = min_x + datetime.timedelta(hours=1)

users = ['nick', 'shawn', 'stacey', 'tim', 'danny']
model_versions = ['v1', 'v2']

num_requests = 10000

def random_datetime_in_range() -> datetime.datetime:
    min_timestamp = min_x.timestamp()
    max_timestamp = max_x.timestamp()
    random_timestamp = random.uniform(min_timestamp, max_timestamp)
    return datetime.datetime.fromtimestamp(random_timestamp)

def random_suggestion(n: int) -> str:
    result = ''
    for i in range(n):
        result += random.choice('abcdefghijklmnopqrstuvwxyz')
    return result


items = weave.save(
    ops_arrow.to_arrow([
        {
            "pred": random_suggestion(7),
            "time": random_datetime_in_range(),
            "user": random.choice(users),
            "version": random.choice(model_versions),
            "accepted": random.choice([1, 1, 1, 0, 0]),
        } 
        for _ in range(num_requests)
    ])
)

In [ ]:
weave.panels.Board(
    vars={
        'predictions': items,
        'x_min': weave.ops.date_parse(str(min_x)),
        'x_max': weave.ops.date_parse(str(max_x))
    },
    panels=[
        weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["time"],
                            label=lambda item: item["version"],
                            agg=lambda preds: preds.count(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="bar",
                            axis_labels={"y": "num_preds"},
                        ),
            layout=weave.panels.BoardPanelLayout(x=0, y=0, w=12, h=6)
        ),
        weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["time"],
                            label=lambda item: item["user"],
                            agg=lambda preds: preds.count(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="bar",
                            axis_labels={"y": "num_preds"},
                        ),
            layout=weave.panels.BoardPanelLayout(x=12, y=0, w=12, h=6)
        ),
        weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["time"],
                            label=lambda item: item["version"],
                            agg=lambda preds: preds["accepted"].sum(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="line",
                            axis_labels={"y": "acceptance_rate"},
                        ),
            layout=weave.panels.BoardPanelLayout(x=0, y=0, w=12, h=6)
        ),
        weave.panels.BoardPanel(
            lambda predictions, x_min, x_max: wandb.TimeSeries(
                            predictions,
                            x=lambda item: item["time"],
                            label=lambda item: item["version"],
                            agg=lambda preds: preds["accepted"].sum() / preds.count(),
                            min_x=x_min,
                            max_x=x_max,
                            mark="line",
                            axis_labels={"y": "acceptance_rate"},
                        ),
            layout=weave.panels.BoardPanelLayout(x=12, y=0, w=12, h=6)
        )
    ]
)